# Spacy NER HTML out

Fichier pris en entrée :
- clean_{date}.json -> provient du notebook **nettoyeur**

Fichiers de sortie produits par le notebook :
- NLP_{date}_{type_pipeline}_{nombre_resultats}.html -> **fichier HTML des résultats de la NER**

In [1]:
import pandas as pd
import spacy
from spacy import displacy
import fr_core_news_lg # md, sm
nlp = fr_core_news_lg.load()
from datetime import date
import time
import json


In [2]:
# Variables à changer par l'utilisateur
source_json = 'results/clean_w_names_24-01-23.json'
pipeline = "fr_core_news_lg" # /!\ PENSER À CHANGER PLUS HAUT

# Importation des données depuis le JSON
df = pd.read_json(source_json, orient='index')

# Nombre de résultats dans la réponse produite, varie en fonction des essais
# nb_a_traiter = len(df.index)
nb_a_traiter = 10

# Variable permettant de compter puis d'inscrire le nombre de "personnes" détéctées par la NER dans le nom du fichier de sortie final (pour contrôle)
count_names = 0

In [3]:
# Définition de l'en tête général
date = time.strftime("%d-%m-%y")

# HTML description des labels
labels_description = """
<p><strong>PERSON:</strong> People, including fictional.&emsp;<strong>NORP:</strong> Nationalities or religious or political groups.&emsp;
<strong>FAC:</strong> Buildings, airports, highways, bridges, etc.&emsp;<strong>ORG:</strong> Companies, agencies, institutions, etc.&emsp;
<strong>GPE:</strong> Countries, cities, states.&emsp;<strong>LOC:</strong> Non-GPE locations, mountain ranges, bodies of water.&emsp;
<strong>PRODUCT:</strong> Objects, vehicles, foods, etc. (Not services.)&emsp;<strong>EVENT:</strong> Named hurricanes, battles, wars, sports events, etc.&emsp;
<strong>WORK_OF_ART:</strong> Titles of books, songs, etc.&emsp;<strong>LAW:</strong> Named documents made into laws.&emsp;<strong>LANGUAGE:</strong> Any named language.&emsp;
<strong>DATE:</strong> Absolute or relative dates or periods.&emsp;<strong>TIME:</strong> Times smaller than a day.&emsp;<strong>PERCENT:</strong> Percentage, including "%".&emsp;
<strong>MONEY:</strong> Monetary values, including unit.&emsp;<strong>QUANTITY:</strong> Measurements, as of weight or distance.&emsp;<strong>ORDINAL:</strong> "first", "second", etc.&emsp;
<strong>CARDINAL:</strong> Numerals that do not fall under another type.</p>
"""

heading = f"""
    <p><strong>date:</strong> {date}</p>
    <p><strong>source:</strong> {source_json}</p>
    <p><strong>pipeline:</strong> {pipeline}</p>
    <p><strong>quantity processessed:</strong> {nb_a_traiter}</p>
    <hr>
    {labels_description}
    """

# (Re)création d'un fichier de sortie propre + en tête
nom_fichier = f"results/spacy_NER_{pipeline[-2:]}_{nb_a_traiter}"

with open(f"{nom_fichier}.html", "w") as fichier:
    fichier.write(heading)

In [4]:
# Personnalisation des couleurs du rendu
colors = {
    "PER": "#97C7E8",
    "ORG": "#A4DBA4",
    "GPE": "#F2937C",
    "LOC": "#AE9DF2",
    "EVENT": "#E8BC76",
    "WORK_OF_ART": "#DB99DB",
    "MISC": "#F2A99D",
    "DATE": "#A7F2BD",
    "ORDINAL": "#E8D3A2",
    "CARDINAL": "#E8D3A2",
}

In [5]:
lists = []

# Boucle de constitution : export PER + HTML
# for i in range(nb_a_traiter): # range(len(df.index))
for i in range(nb_a_traiter):
    # NLP par spacy --------------------------------------------------------------------------
    title = df.loc[i, 'area_title']
    text = df.loc[i, 'area_text']
    extracted_names = df.loc[i, 'extracted_names']
    doc = nlp(text)

    # Constitution de l'HTML de contrôle du NLP ----------------------------------------------
    sentence_tokens = len([[token.text for token in sent] for sent in doc.sents])

    # Génération du rendu displacy
    html = displacy.render(doc, style="ent", jupyter=False, page=True, options={"colors": colors})

    # Définition de l'en tête pour chaque résultat
    headings = f"""
    <hr>
    <p><strong>index:</strong> {i}</p>
    <p><strong>title:</strong> {title}</p>
    <p><strong>number of sentences:</strong> {sentence_tokens}</p>
    """

    # Inscription de l'en tête + inscription de résultat
    with open(f"{nom_fichier}.html", 'a') as fichier:
        fichier.write(headings)
        fichier.write(str(html))